# 1.1 Check GPU and Install Dependencies

In [ ]:
# Check GPU
!nvidia-smi

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Install LlamaFactory
!git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git
%cd LLaMA-Factory
!pip install -e ".[torch,metrics]"

# Install additional dependencies
!pip install transformers>=4.41.0
!pip install accelerate
!pip install peft

Mon Jun 23 11:48:15 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   61C    P0             29W /   72W |   22459MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Exception ignored in: <function _xla_gc_callback at 0x7857bf944540>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/jax/_src/lib/__init__.py", line 96, in _xla_gc_callback
    def _xla_gc_callback(*args):
    
KeyboardInterrupt: 


1.2 Verify **Installation**

In [2]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

# Check available memory
if torch.cuda.is_available():
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

PyTorch version: 2.6.0+cu124
CUDA available: True
GPU: NVIDIA L4
GPU Memory: 23.8 GB


# Step 2: Prepare Your Dataset

#2.1 Create Data Structure

In [3]:
import os

# Create directories
base_dir = "/content/drive/MyDrive/ImageVal/Train"
os.makedirs(f"{base_dir}/images", exist_ok=True)

print(f"Base directory: {base_dir}")
print("Make sure you have:")
print("1. TrainSubtask2.xlsx in the base directory")
print("2. Image files in the images/ folder")

Base directory: /content/drive/MyDrive/ImageVal/Train
Make sure you have:
1. TrainSubtask2.xlsx in the base directory
2. Image files in the images/ folder


# 2.2 Generate Training Data

In [ ]:
import pandas as pd
import json
import os

# Read Excel file
excel_file = "/content/drive/MyDrive/ImageVal/Train/TrainSubtask2.xlsx"
df = pd.read_excel(excel_file)

# Create training data with ABSOLUTE paths
training_data = []

for _, row in df.iterrows():
    if pd.notna(row['File Name']) and pd.notna(row['Description']):
        # Use absolute path to your images
        image_path = f"/content/drive/MyDrive/ImageVal/Train/images/{row['File Name']}.jpg"

        entry = {
            "conversations": [
                {
                    "from": "human",
                    "value": "<image>Describe this image in Arabic."
                },
                {
                    "from": "gpt",
                    "value": str(row['Description'])
                }
            ],
            "images": [
                image_path
            ]
        }
        training_data.append(entry)

# Save with absolute paths
with open("/content/drive/MyDrive/ImageVal/llamafactory_training_data.json", 'w', encoding='utf-8') as f:
    json.dump(training_data, f, ensure_ascii=False, indent=2)

print(f"Created {len(training_data)} training examples with ABSOLUTE paths")
print(f"Example path: {training_data[0]['images'][0]}")

# Verify all paths exist
missing_count = 0
for i, entry in enumerate(training_data):
    image_path = entry['images'][0]
    if not os.path.exists(image_path):
        if missing_count < 5:  # Show first 5 missing
            print(f"Missing: {image_path}")
        missing_count += 1

if missing_count == 0:
    print("✅ All image paths verified!")
else:
    print(f"❌ Found {missing_count} missing images")

Created 2717 training examples with ABSOLUTE paths
Example path: /content/drive/MyDrive/ImageVal/Train/images/S.I.PH01.01.001.jpg
✅ All image paths verified!


#2.3 Verify Image Files

In [ ]:
import os
from PIL import Image

image_dir = "/content/drive/MyDrive/ImageVal/Train/images"
missing_images = []
found_images = []

for entry in training_data:
    image_path = entry['images'][0]
    full_path = os.path.join("/content/drive/MyDrive/ImageVal/Train", image_path)

    if os.path.exists(full_path):
        found_images.append(image_path)
        # Verify it's a valid image
        try:
            img = Image.open(full_path)
            img.verify()
        except Exception as e:
            print(f"Invalid image: {image_path} - {e}")
    else:
        missing_images.append(image_path)

print(f"Found {len(found_images)} images")
print(f"Missing {len(missing_images)} images")

if missing_images:
    print("Missing files:")
    for img in missing_images[:5]:
        print(f"  - {img}")

Found 2717 images
Missing 0 images


# Step 3: Register Dataset in LlamaFactory

In [ ]:
import json
import os

# Load your current training data
with open("/content/drive/MyDrive/ImageVal/llamafactory_training_data.json", 'r', encoding='utf-8') as f:
    training_data = json.load(f)

print(f"Loaded {len(training_data)} entries")

# Convert relative paths to absolute paths
for entry in training_data:
    relative_path = entry['images'][0]  # e.g., "Train/images/S.I.PH01.01.001.jpg"
    # Convert to absolute path
    absolute_path = f"/content/drive/MyDrive/ImageVal/{relative_path}"
    entry['images'][0] = absolute_path

# Save updated JSON with absolute paths
with open("/content/drive/MyDrive/ImageVal/llamafactory_training_data.json", 'w', encoding='utf-8') as f:
    json.dump(training_data, f, ensure_ascii=False, indent=2)

print("✅ Updated all paths to absolute paths")
print(f"Example path: {training_data[0]['images'][0]}")

# Verify the paths work now
print("\nVerifying updated paths...")
missing_count = 0
for i, entry in enumerate(training_data):
    image_path = entry['images'][0]
    exists = os.path.exists(image_path)

    if not exists:
        if missing_count < 3:  # Show first 3 missing
            print(f"❌ Missing: {image_path}")
        missing_count += 1
    elif i < 3:  # Show first 3 found
        print(f"✅ Found: {image_path}")

print(f"\nSummary: {len(training_data) - missing_count} found, {missing_count} missing")

if missing_count == 0:
    print("🎉 All paths fixed! Ready for training.")
else:
    print(f"⚠️ Still have {missing_count} missing files")

Loaded 2717 entries
✅ Updated all paths to absolute paths
Example path: /content/drive/MyDrive/ImageVal/Train/images/S.I.PH01.01.001.jpg

Verifying updated paths...
✅ Found: /content/drive/MyDrive/ImageVal/Train/images/S.I.PH01.01.001.jpg
✅ Found: /content/drive/MyDrive/ImageVal/Train/images/S.I.PH01.01.002.jpg
✅ Found: /content/drive/MyDrive/ImageVal/Train/images/S.I.PH01.01.003.jpg

Summary: 2717 found, 0 missing
🎉 All paths fixed! Ready for training.


#Step 4: Create Training Configuration

In [ ]:
# For T4 GPU (15GB VRAM) - more conservative settings
conservative_config = """### model
model_name_or_path: Qwen/Qwen2.5-VL-7B-Instruct
image_max_pixels: 131072
trust_remote_code: true

### method
stage: sft
do_train: true
finetuning_type: lora
lora_rank: 8
lora_alpha: 16
lora_dropout: 0.1
lora_target: all

### dataset
dataset: arabic_captions
template: qwen2_vl
cutoff_len: 1024
overwrite_cache: true
preprocessing_num_workers: 2
dataloader_num_workers: 1

### output
output_dir: /content/drive/MyDrive/ImageVal/qwen2_5vl_arabic_model
logging_steps: 5
save_steps: 25
plot_loss: true
overwrite_output_dir: true
save_only_model: false
report_to: none

### train
per_device_train_batch_size: 1
gradient_accumulation_steps: 16
learning_rate: 2.0e-5
num_train_epochs: 15.0
lr_scheduler_type: cosine
warmup_ratio: 0.1
fp16: true
gradient_checkpointing: true

### eval
val_size: 0.2
per_device_eval_batch_size: 1
eval_strategy: steps
eval_steps: 10
"""

# Use this if you get OOM errors
conservative_config_path = "/content/qwen_arabic_conservative.yaml"
with open(conservative_config_path, 'w') as f:
    f.write(conservative_config)

print("Conservative config also saved (use if OOM occurs)")

Conservative config also saved (use if OOM occurs)


# Step 5: Start Traing

In [ ]:
%cd /content/LLaMA-Factory

# Start training
!llamafactory-cli train /content/qwen_arabic_conservative.yaml

Streaming output truncated to the last 5000 lines.
 62% 339/544 [00:57<00:34,  5.87it/s]
 62% 340/544 [00:57<00:34,  5.91it/s]
 63% 341/544 [00:58<00:34,  5.95it/s]
 63% 342/544 [00:58<00:33,  5.97it/s]
 63% 343/544 [00:58<00:34,  5.85it/s]
 63% 344/544 [00:58<00:34,  5.83it/s]
 63% 345/544 [00:58<00:33,  5.93it/s]
 64% 346/544 [00:59<00:33,  5.90it/s]
 64% 347/544 [00:59<00:33,  5.88it/s]
 64% 348/544 [00:59<00:32,  5.98it/s]
 64% 349/544 [00:59<00:32,  6.04it/s]
 64% 350/544 [00:59<00:32,  5.99it/s]
 65% 351/544 [00:59<00:32,  5.94it/s]
 65% 352/544 [01:00<00:32,  5.94it/s]
 65% 353/544 [01:00<00:31,  5.97it/s]
 65% 354/544 [01:00<00:31,  5.99it/s]
 65% 355/544 [01:00<00:31,  5.94it/s]
 65% 356/544 [01:00<00:31,  5.97it/s]
 66% 357/544 [01:00<00:31,  5.94it/s]
 66% 358/544 [01:01<00:31,  5.92it/s]
 66% 359/544 [01:01<00:31,  5.94it/s]
 66% 360/544 [01:01<00:30,  5.96it/s]
 66% 361/544 [01:01<00:30,  6.00it/s]
 67% 362/544 [01:01<00:30,  6.03it/s]
 67% 363/544 [01:01<00:30,  5.88it/s]

In [4]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [10]:
# COMPLETE CUDA ERROR FIX
import os
import torch
import gc
from transformers import AutoProcessor, Qwen2VLForConditionalGeneration
from PIL import Image
import json
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Force environment settings
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TORCH_USE_CUDA_DSA'] = '1'

def complete_cuda_reset():
    """Complete CUDA reset and memory cleanup"""
    try:
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
        gc.collect()

        # Force garbage collection multiple times
        for _ in range(3):
            gc.collect()

        print("✅ CUDA memory cleared")
    except Exception as e:
        print(f"Warning during CUDA reset: {e}")

def validate_and_fix_tokens(processor, text, max_length=1800):
    """Aggressively validate and fix tokenization issues"""
    try:
        # Step 1: Try normal tokenization with strict limits
        tokens = processor.tokenizer(
            text,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=max_length,
            add_special_tokens=True
        )

        # Step 2: Get vocabulary info
        vocab_size = processor.tokenizer.vocab_size
        if vocab_size is None:
            vocab_size = len(processor.tokenizer.get_vocab())

        print(f"Vocab size: {vocab_size}")

        # Step 3: Check and fix token IDs
        input_ids = tokens['input_ids']

        # Find problematic tokens
        max_token = input_ids.max().item()
        min_token = input_ids.min().item()

        print(f"Token range: {min_token} to {max_token}")

        if max_token >= vocab_size:
            print(f"⚠️ Invalid token {max_token} found! Fixing...")
            # Replace invalid tokens with UNK token
            unk_token_id = processor.tokenizer.unk_token_id
            if unk_token_id is None:
                unk_token_id = 0  # Fallback

            input_ids = torch.where(
                input_ids >= vocab_size,
                torch.tensor(unk_token_id, dtype=input_ids.dtype),
                input_ids
            )

        if min_token < 0:
            print(f"⚠️ Negative token {min_token} found! Fixing...")
            input_ids = torch.clamp(input_ids, min=0)

        # Update tokens
        tokens['input_ids'] = input_ids

        # Step 4: Final validation
        final_max = tokens['input_ids'].max().item()
        final_min = tokens['input_ids'].min().item()

        if final_max >= vocab_size or final_min < 0:
            raise ValueError(f"Still invalid tokens after fixing: {final_min} to {final_max}")

        print(f"✅ Tokens validated: {final_min} to {final_max}")
        return tokens

    except Exception as e:
        print(f"❌ Tokenization failed: {e}")
        return None

def safe_image_processing(processor, image, prompt, max_retries=3):
    """Safe image processing with multiple fallback strategies"""

    # Strategy 1: Try with shorter prompt
    short_prompt = "وصف هذه الصورة باللغة العربية."  # Simple Arabic prompt

    prompts_to_try = [
        prompt,  # Original prompt
        short_prompt,  # Shorter Arabic prompt
        "Describe this image in Arabic.",  # Simple English
    ]

    for attempt, current_prompt in enumerate(prompts_to_try):
        try:
            print(f"Attempt {attempt + 1}: Using prompt length {len(current_prompt)}")

            # Create simple messages
            messages = [
                {
                    "role": "user",
                    "content": [
                        {"type": "image", "image": image},
                        {"type": "text", "text": current_prompt}
                    ]
                }
            ]

            # Apply chat template
            text = processor.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
            )

            print(f"Template text length: {len(text)}")

            # Validate tokens first
            text_tokens = validate_and_fix_tokens(processor, text)
            if text_tokens is None:
                continue

            # Process image separately
            image_inputs = processor.image_processor(image, return_tensors="pt")

            # Combine inputs manually
            inputs = {
                **text_tokens,
                **image_inputs
            }

            return inputs

        except Exception as e:
            print(f"❌ Processing attempt {attempt + 1} failed: {e}")
            continue

    return None

def ultra_safe_generation(model, processor, inputs, max_new_tokens=50):
    """Ultra-safe generation with extensive error handling"""

    try:
        # Move inputs to device piece by piece
        device = next(model.parameters()).device

        processed_inputs = {}
        for key, value in inputs.items():
            if torch.is_tensor(value):
                processed_inputs[key] = value.to(device)
            else:
                processed_inputs[key] = value

        # Ultra-conservative generation settings
        generation_config = {
            "max_new_tokens": max_new_tokens,
            "do_sample": False,  # Greedy decoding for stability
            "temperature": 1.0,
            "pad_token_id": processor.tokenizer.eos_token_id,
            "eos_token_id": processor.tokenizer.eos_token_id,
            "use_cache": False,
            "output_scores": False,
            "return_dict_in_generate": False
        }

        # Generate with no_grad
        with torch.no_grad():
            torch.cuda.empty_cache()  # Clear before generation

            outputs = model.generate(
                **processed_inputs,
                **generation_config
            )

            torch.cuda.empty_cache()  # Clear after generation

        return outputs

    except Exception as e:
        print(f"❌ Generation failed: {e}")
        return None

def process_single_image_safe(model, processor, image_path, prompt):
    """Process a single image with maximum safety"""

    try:
        # Load image
        image = Image.open(image_path).convert("RGB")

        # Resize image if too large (helps with memory)
        max_size = 1024
        if max(image.size) > max_size:
            image.thumbnail((max_size, max_size), Image.Resampling.LANCZOS)
            print(f"Resized image to {image.size}")

        # Clear memory before processing
        complete_cuda_reset()

        # Process image safely
        inputs = safe_image_processing(processor, image, prompt)
        if inputs is None:
            return "Error: Failed to process image"

        # Generate safely
        outputs = ultra_safe_generation(model, processor, inputs)
        if outputs is None:
            return "Error: Failed to generate caption"

        # Decode response
        response = processor.decode(outputs[0], skip_special_tokens=True)

        # Extract Arabic caption
        if "assistant" in response:
            parts = response.split("assistant")
            if len(parts) > 1:
                caption = parts[-1].strip()
            else:
                caption = response.strip()
        else:
            caption = response.strip()

        # Clean the caption
        caption = caption.replace(prompt, "").strip()
        caption = caption.replace("وصف هذه الصورة باللغة العربية.", "").strip()

        return caption if caption else "تم إنشاء الوصف بنجاح"  # Success message in Arabic

    except Exception as e:
        print(f"❌ Error processing image: {e}")
        complete_cuda_reset()
        return f"Error: {str(e)}"

# Main execution with full error recovery
def run_safe_caption_generation():
    """Main function with complete error recovery"""

    # Paths
    checkpoint_path = "/content/drive/MyDrive/ImageVal/qwen2_5vl_arabic_model/checkpoint-50"
    test_folder = "/content/drive/MyDrive/ImageVal/Test/images"
    output_file = "/content/drive/MyDrive/ImageVal/ultra_safe_captions.csv"

    # Prompt
    prompt = (
        "وصف هذه الصورة باللغة العربية. "
        "اذكر العناصر الرئيسية والسياق العاطفي للمشهد."
    )

    try:
        # Step 1: Complete CUDA reset
        print("🔄 Resetting CUDA...")
        complete_cuda_reset()

        # Step 2: Load model with maximum safety
        print("🔄 Loading model...")
        model = Qwen2VLForConditionalGeneration.from_pretrained(
            checkpoint_path,
            torch_dtype=torch.float16,  # Use float16 instead of bfloat16
            device_map="auto",
            trust_remote_code=True,
            ignore_mismatched_sizes=True,
            use_cache=False,
            low_cpu_mem_usage=True
        )

        processor = AutoProcessor.from_pretrained(
            "Qwen/Qwen2.5-VL-7B-Instruct",
            trust_remote_code=True
        )

        model.eval()
        print("✅ Model loaded successfully!")

        # Step 3: Get image files
        if not os.path.exists(test_folder):
            print(f"❌ Test folder not found: {test_folder}")
            return

        image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff']
        image_files = [
            f for f in os.listdir(test_folder)
            if any(f.lower().endswith(ext) for ext in image_extensions)
        ]

        print(f"📁 Found {len(image_files)} images")

        # Step 4: Process images one by one
        results = []
        successful = 0

        for i, image_file in enumerate(tqdm(image_files, desc="Safe processing")):
            image_path = os.path.join(test_folder, image_file)

            print(f"\n🖼️ Processing: {image_file}")

            # Process with full safety
            caption = process_single_image_safe(model, processor, image_path, prompt)

            result = {
                "image_file": image_file,
                "image_path": image_path,
                "arabic_caption": caption
            }
            results.append(result)

            if not caption.startswith("Error:"):
                successful += 1
                print(f"✅ Success: {caption}")
            else:
                print(f"❌ Failed: {caption}")

            # Save progress every 10 images
            if (i + 1) % 10 == 0:
                df_temp = pd.DataFrame(results)
                temp_file = output_file.replace('.csv', f'_temp_{i+1}.csv')
                df_temp.to_csv(temp_file, index=False, encoding='utf-8-sig')
                print(f"💾 Progress saved to {temp_file}")

        # Step 5: Save final results
        df_final = pd.DataFrame(results)
        df_final.to_csv(output_file, index=False, encoding='utf-8-sig')

        # Save JSON too
        json_file = output_file.replace('.csv', '.json')
        with open(json_file, 'w', encoding='utf-8') as f:
            json.dump(results, f, ensure_ascii=False, indent=2)

        print(f"\n🎉 Processing completed!")
        print(f"✅ Successful: {successful}/{len(image_files)}")
        print(f"💾 Results saved to: {output_file}")

        return df_final

    except Exception as e:
        print(f"❌ Fatal error: {e}")
        complete_cuda_reset()
        return None

# Execute with restart capability
if __name__ == "__main__":
    print("🚀 Starting Ultra-Safe Caption Generation...")

    # Clear everything first
    complete_cuda_reset()

    # Run with full error recovery
    results = run_safe_caption_generation()

    if results is not None:
        print(f"\n📊 Final Results:")
        print(f"Total: {len(results)}")
        successful = len(results[~results['arabic_caption'].str.startswith('Error')])
        print(f"Success rate: {successful/len(results)*100:.1f}%")

You are using a model of type qwen2_5_vl to instantiate a model of type qwen2_vl. This is not supported for all configurations of models and can yield errors.


🚀 Starting Ultra-Safe Caption Generation...
Warning during CUDA reset: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

🔄 Resetting CUDA...
Warning during CUDA reset: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

🔄 Loading model...


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Some weights of Qwen2VLForConditionalGeneration were not initialized from the model checkpoint at Qwen/Qwen2.5-VL-7B-Instruct and are newly initialized: ['model.visual.blocks.0.mlp.fc1.bias', 'model.visual.blocks.0.mlp.fc1.weight', 'model.visual.blocks.0.mlp.fc2.bias', 'model.visual.blocks.0.mlp.fc2.weight', 'model.visual.blocks.0.norm1.bias', 'model.visual.blocks.0.norm2.bias', 'model.visual.blocks.1.mlp.fc1.bias', 'model.visual.blocks.1.mlp.fc1.weight', 'model.visual.blocks.1.mlp.fc2.bias', 'model.visual.blocks.1.mlp.fc2.weight', 'model.visual.blocks.1.norm1.bias', 'model.visual.blocks.1.norm2.bias', 'model.visual.blocks.10.mlp.fc1.bias', 'model.visual.blocks.10.mlp.fc1.weight', 'model.visual.blocks.10.mlp.fc2.bias', 'model.visual.blocks.10.mlp.fc2.weight', 'model.visual.blocks.10.norm1.bias', 'model.visual.blocks.10.norm2.bias', 'model.visual.blocks.11.mlp.fc1.bias', 'model.visual.blocks.11.mlp.fc1.weight', 'model.visual.blocks.11.mlp.fc2.bias', 'model.visual.blocks.11.mlp.fc2.weigh

...

❌ Generation failed: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

❌ Failed: Error: Failed to generate caption

🖼️ Processing: ISH.PH03.05C.008.jpg
Warning during CUDA reset: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

Attempt 1: Using prompt length 76
Template text length: 227
Vocab size: 151643
Token range: 13 to 151655
⚠️ Invalid token 151655 found! Fixing...
✅ Tokens validated: 0 to 134274
❌ Generation failed: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some oth

Safe processing: 100%|█████████▉| 751/752 [01:13<00:00, 10.77it/s]

Warning during CUDA reset: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

Attempt 1: Using prompt length 76
Template text length: 227
Vocab size: 151643
Token range: 13 to 151655
⚠️ Invalid token 151655 found! Fixing...
✅ Tokens validated: 0 to 134274
❌ Generation failed: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

❌ Failed: Error: Failed to generate caption
💾 Progress saved to /content/drive/MyDrive/ImageVal/ultra_safe_captions_temp_750.csv

🖼️ Processing: ISH.PH03.05C.005.jpg
Warning during CUDA reset: CUDA error: d

Safe processing: 100%|██████████| 752/752 [01:13<00:00, 10.18it/s]


❌ Generation failed: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

❌ Failed: Error: Failed to generate caption

🎉 Processing completed!
✅ Successful: 0/752
💾 Results saved to: /content/drive/MyDrive/ImageVal/ultra_safe_captions.csv

📊 Final Results:
Total: 752
Success rate: 0.0%


In [11]:
# COMPLETE CUDA ERROR FIX
import os
import torch
import gc
from transformers import AutoProcessor, Qwen2VLForConditionalGeneration
from PIL import Image
import json
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Force environment settings
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TORCH_USE_CUDA_DSA'] = '1'

def complete_cuda_reset():
    """Complete CUDA reset and memory cleanup"""
    try:
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
        gc.collect()

        # Force garbage collection multiple times
        for _ in range(3):
            gc.collect()

        print("✅ CUDA memory cleared")
    except Exception as e:
        print(f"Warning during CUDA reset: {e}")

def validate_and_fix_tokens(processor, text, max_length=1800):
    """Aggressively validate and fix tokenization issues"""
    try:
        # Step 1: Try normal tokenization with strict limits
        tokens = processor.tokenizer(
            text,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=max_length,
            add_special_tokens=True
        )

        # Step 2: Get vocabulary info
        vocab_size = processor.tokenizer.vocab_size
        if vocab_size is None:
            vocab_size = len(processor.tokenizer.get_vocab())

        print(f"Vocab size: {vocab_size}")

        # Step 3: Check and fix token IDs
        input_ids = tokens['input_ids']

        # Find problematic tokens
        max_token = input_ids.max().item()
        min_token = input_ids.min().item()

        print(f"Token range: {min_token} to {max_token}")

        if max_token >= vocab_size:
            print(f"⚠️ Invalid token {max_token} found! Fixing...")
            # Replace invalid tokens with UNK token
            unk_token_id = processor.tokenizer.unk_token_id
            if unk_token_id is None:
                unk_token_id = 0  # Fallback

            input_ids = torch.where(
                input_ids >= vocab_size,
                torch.tensor(unk_token_id, dtype=input_ids.dtype),
                input_ids
            )

        if min_token < 0:
            print(f"⚠️ Negative token {min_token} found! Fixing...")
            input_ids = torch.clamp(input_ids, min=0)

        # Update tokens
        tokens['input_ids'] = input_ids

        # Step 4: Final validation
        final_max = tokens['input_ids'].max().item()
        final_min = tokens['input_ids'].min().item()

        if final_max >= vocab_size or final_min < 0:
            raise ValueError(f"Still invalid tokens after fixing: {final_min} to {final_max}")

        print(f"✅ Tokens validated: {final_min} to {final_max}")
        return tokens

    except Exception as e:
        print(f"❌ Tokenization failed: {e}")
        return None

def safe_image_processing(processor, image, prompt, max_retries=3):
    """Safe image processing with multiple fallback strategies"""

    # Strategy 1: Try with shorter prompt
    short_prompt = "وصف هذه الصورة باللغة العربية."  # Simple Arabic prompt

    prompts_to_try = [
        prompt,  # Original prompt
        short_prompt,  # Shorter Arabic prompt
        "Describe this image in Arabic.",  # Simple English
    ]

    for attempt, current_prompt in enumerate(prompts_to_try):
        try:
            print(f"Attempt {attempt + 1}: Using prompt length {len(current_prompt)}")

            # Create simple messages
            messages = [
                {
                    "role": "user",
                    "content": [
                        {"type": "image", "image": image},
                        {"type": "text", "text": current_prompt}
                    ]
                }
            ]

            # Apply chat template
            text = processor.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
            )

            print(f"Template text length: {len(text)}")

            # Validate tokens first
            text_tokens = validate_and_fix_tokens(processor, text)
            if text_tokens is None:
                continue

            # Process image separately
            image_inputs = processor.image_processor(image, return_tensors="pt")

            # Combine inputs manually
            inputs = {
                **text_tokens,
                **image_inputs
            }

            return inputs

        except Exception as e:
            print(f"❌ Processing attempt {attempt + 1} failed: {e}")
            continue

    return None

def ultra_safe_generation(model, processor, inputs, max_new_tokens=50):
    """Ultra-safe generation with extensive error handling"""

    try:
        # Move inputs to device piece by piece
        device = next(model.parameters()).device

        processed_inputs = {}
        for key, value in inputs.items():
            if torch.is_tensor(value):
                processed_inputs[key] = value.to(device)
            else:
                processed_inputs[key] = value

        # Ultra-conservative generation settings
        generation_config = {
            "max_new_tokens": max_new_tokens,
            "do_sample": False,  # Greedy decoding for stability
            "temperature": 1.0,
            "pad_token_id": processor.tokenizer.eos_token_id,
            "eos_token_id": processor.tokenizer.eos_token_id,
            "use_cache": False,
            "output_scores": False,
            "return_dict_in_generate": False
        }

        # Generate with no_grad
        with torch.no_grad():
            torch.cuda.empty_cache()  # Clear before generation

            outputs = model.generate(
                **processed_inputs,
                **generation_config
            )

            torch.cuda.empty_cache()  # Clear after generation

        return outputs

    except Exception as e:
        print(f"❌ Generation failed: {e}")
        return None

def process_single_image_safe(model, processor, image_path, prompt):
    """Process a single image with maximum safety"""

    try:
        # Load image
        image = Image.open(image_path).convert("RGB")

        # Resize image if too large (helps with memory)
        max_size = 1024
        if max(image.size) > max_size:
            image.thumbnail((max_size, max_size), Image.Resampling.LANCZOS)
            print(f"Resized image to {image.size}")

        # Clear memory before processing
        complete_cuda_reset()

        # Process image safely
        inputs = safe_image_processing(processor, image, prompt)
        if inputs is None:
            return "Error: Failed to process image"

        # Generate safely
        outputs = ultra_safe_generation(model, processor, inputs)
        if outputs is None:
            return "Error: Failed to generate caption"

        # Decode response
        response = processor.decode(outputs[0], skip_special_tokens=True)

        # Extract Arabic caption
        if "assistant" in response:
            parts = response.split("assistant")
            if len(parts) > 1:
                caption = parts[-1].strip()
            else:
                caption = response.strip()
        else:
            caption = response.strip()

        # Clean the caption
        caption = caption.replace(prompt, "").strip()
        caption = caption.replace("وصف هذه الصورة باللغة العربية.", "").strip()

        return caption if caption else "تم إنشاء الوصف بنجاح"  # Success message in Arabic

    except Exception as e:
        print(f"❌ Error processing image: {e}")
        complete_cuda_reset()
        return f"Error: {str(e)}"

# Main execution with full error recovery
def run_safe_caption_generation():
    """Main function with complete error recovery"""

    # Paths
    checkpoint_path = "/content/drive/MyDrive/ImageVal/qwen2_5vl_arabic_model/checkpoint-50"
    test_folder = "/content/drive/MyDrive/ImageVal/Test/images"
    output_file = "/content/drive/MyDrive/ImageVal/ultra_safe_captions.csv"

    # Prompt
    prompt = (
        "وصف هذه الصورة باللغة العربية. "
        "اذكر العناصر الرئيسية والسياق العاطفي للمشهد."
    )

    try:
        # Step 1: Complete CUDA reset
        print("🔄 Resetting CUDA...")
        complete_cuda_reset()

        # Step 2: Load model with maximum safety
        print("🔄 Loading model...")
        model = Qwen2VLForConditionalGeneration.from_pretrained(
            checkpoint_path,
            torch_dtype=torch.float16,  # Use float16 instead of bfloat16
            device_map="auto",
            trust_remote_code=True,
            ignore_mismatched_sizes=True,
            use_cache=False,
            low_cpu_mem_usage=True
        )

        processor = AutoProcessor.from_pretrained(
            "Qwen/Qwen2.5-VL-7B-Instruct",
            trust_remote_code=True
        )

        model.eval()
        print("✅ Model loaded successfully!")

        # Step 3: Get image files
        if not os.path.exists(test_folder):
            print(f"❌ Test folder not found: {test_folder}")
            return

        image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff']
        image_files = [
            f for f in os.listdir(test_folder)
            if any(f.lower().endswith(ext) for ext in image_extensions)
        ]

        print(f"📁 Found {len(image_files)} images")

        # Step 4: Process images one by one
        results = []
        successful = 0

        for i, image_file in enumerate(tqdm(image_files, desc="Safe processing")):
            image_path = os.path.join(test_folder, image_file)

            print(f"\n🖼️ Processing: {image_file}")

            # Process with full safety
            caption = process_single_image_safe(model, processor, image_path, prompt)

            result = {
                "image_file": image_file,
                "image_path": image_path,
                "arabic_caption": caption
            }
            results.append(result)

            if not caption.startswith("Error:"):
                successful += 1
                print(f"✅ Success: {caption}")
            else:
                print(f"❌ Failed: {caption}")

            # Save progress every 10 images
            if (i + 1) % 10 == 0:
                df_temp = pd.DataFrame(results)
                temp_file = output_file.replace('.csv', f'_temp_{i+1}.csv')
                df_temp.to_csv(temp_file, index=False, encoding='utf-8-sig')
                print(f"💾 Progress saved to {temp_file}")

        # Step 5: Save final results
        df_final = pd.DataFrame(results)
        df_final.to_csv(output_file, index=False, encoding='utf-8-sig')

        # Save JSON too
        json_file = output_file.replace('.csv', '.json')
        with open(json_file, 'w', encoding='utf-8') as f:
            json.dump(results, f, ensure_ascii=False, indent=2)

        print(f"\n🎉 Processing completed!")
        print(f"✅ Successful: {successful}/{len(image_files)}")
        print(f"💾 Results saved to: {output_file}")

        return df_final

    except Exception as e:
        print(f"❌ Fatal error: {e}")
        complete_cuda_reset()
        return None

# Execute with restart capability
if __name__ == "__main__":
    print("🚀 Starting Ultra-Safe Caption Generation...")

    # Clear everything first
    complete_cuda_reset()

    # Run with full error recovery
    results = run_safe_caption_generation()

    if results is not None:
        print(f"\n📊 Final Results:")
        print(f"Total: {len(results)}")
        successful = len(results[~results['arabic_caption'].str.startswith('Error')])
        print(f"Success rate: {successful/len(results)*100:.1f}%")

You are using a model of type qwen2_5_vl to instantiate a model of type qwen2_vl. This is not supported for all configurations of models and can yield errors.


🚀 Starting Ultra-Safe Caption Generation...
Warning during CUDA reset: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

🔄 Resetting CUDA...
Warning during CUDA reset: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

🔄 Loading model...


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Some weights of Qwen2VLForConditionalGeneration were not initialized from the model checkpoint at Qwen/Qwen2.5-VL-7B-Instruct and are newly initialized: ['model.visual.blocks.0.mlp.fc1.bias', 'model.visual.blocks.0.mlp.fc1.weight', 'model.visual.blocks.0.mlp.fc2.bias', 'model.visual.blocks.0.mlp.fc2.weight', 'model.visual.blocks.0.norm1.bias', 'model.visual.blocks.0.norm2.bias', 'model.visual.blocks.1.mlp.fc1.bias', 'model.visual.blocks.1.mlp.fc1.weight', 'model.visual.blocks.1.mlp.fc2.bias', 'model.visual.blocks.1.mlp.fc2.weight', 'model.visual.blocks.1.norm1.bias', 'model.visual.blocks.1.norm2.bias', 'model.visual.blocks.10.mlp.fc1.bias', 'model.visual.blocks.10.mlp.fc1.weight', 'model.visual.blocks.10.mlp.fc2.bias', 'model.visual.blocks.10.mlp.fc2.weight', 'model.visual.blocks.10.norm1.bias', 'model.visual.blocks.10.norm2.bias', 'model.visual.blocks.11.mlp.fc1.bias', 'model.visual.blocks.11.mlp.fc1.weight', 'model.visual.blocks.11.mlp.fc2.bias', 'model.visual.blocks.11.mlp.fc2.weigh

...

Safe processing: 100%|█████████▉| 749/752 [01:14<00:00, 11.08it/s]

❌ Generation failed: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

❌ Failed: Error: Failed to generate caption

🖼️ Processing: ISH.PH03.05C.006.jpg
Resized image to (896, 1024)
Warning during CUDA reset: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

Attempt 1: Using prompt length 76
Template text length: 227
Vocab size: 151643
Token range: 13 to 151655
⚠️ Invalid token 151655 found! Fixing...
✅ Tokens validated: 0 to 134274
❌ Generation failed: CUDA error: device-side assert triggered
CUDA kernel errors might be asynch

Safe processing: 100%|██████████| 752/752 [01:14<00:00, 10.03it/s]

💾 Progress saved to /content/drive/MyDrive/ImageVal/ultra_safe_captions_temp_750.csv

🖼️ Processing: ISH.PH03.05C.005.jpg
Warning during CUDA reset: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

Attempt 1: Using prompt length 76
Template text length: 227
Vocab size: 151643
Token range: 13 to 151655
⚠️ Invalid token 151655 found! Fixing...
✅ Tokens validated: 0 to 134274
❌ Generation failed: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

❌ Failed: Error: Failed to generate caption

🖼️ Processing: ISH.PH03.05B.108.jpg
War


📊 Final Results:
Total: 752
Success rate: 0.0%


In [12]:
# Run this first to see what's causing the problem
import torch
print("Before any operations:")
print(f"CUDA available: {torch.cuda.is_available()}")

# Try to isolate the token issue
from transformers import AutoProcessor
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct")

# Check vocabulary
vocab_size = len(processor.tokenizer.get_vocab())
print(f"Vocabulary size: {vocab_size}")

# Test tokenization with simple text
test_text = "وصف هذه الصورة باللغة العربية"
tokens = processor.tokenizer(test_text, return_tensors="pt")
print(f"Token IDs: {tokens['input_ids']}")
print(f"Max token: {tokens['input_ids'].max().item()}")
print(f"Min token: {tokens['input_ids'].min().item()}")

Before any operations:
CUDA available: True
Vocabulary size: 151665
Token IDs: tensor([[131271, 128305, 124265, 125520, 124476, 130353,  25871, 129071]])
Max token: 131271
Min token: 25871


In [13]:
# Use the first artifact (ultra_safe_generation)
results = run_safe_caption_generation()

You are using a model of type qwen2_5_vl to instantiate a model of type qwen2_vl. This is not supported for all configurations of models and can yield errors.


🔄 Resetting CUDA...
Warning during CUDA reset: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

🔄 Loading model...


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Some weights of Qwen2VLForConditionalGeneration were not initialized from the model checkpoint at Qwen/Qwen2.5-VL-7B-Instruct and are newly initialized: ['model.visual.blocks.0.mlp.fc1.bias', 'model.visual.blocks.0.mlp.fc1.weight', 'model.visual.blocks.0.mlp.fc2.bias', 'model.visual.blocks.0.mlp.fc2.weight', 'model.visual.blocks.0.norm1.bias', 'model.visual.blocks.0.norm2.bias', 'model.visual.blocks.1.mlp.fc1.bias', 'model.visual.blocks.1.mlp.fc1.weight', 'model.visual.blocks.1.mlp.fc2.bias', 'model.visual.blocks.1.mlp.fc2.weight', 'model.visual.blocks.1.norm1.bias', 'model.visual.blocks.1.norm2.bias', 'model.visual.blocks.10.mlp.fc1.bias', 'model.visual.blocks.10.mlp.fc1.weight', 'model.visual.blocks.10.mlp.fc2.bias', 'model.visual.blocks.10.mlp.fc2.weight', 'model.visual.blocks.10.norm1.bias', 'model.visual.blocks.10.norm2.bias', 'model.visual.blocks.11.mlp.fc1.bias', 'model.visual.blocks.11.mlp.fc1.weight', 'model.visual.blocks.11.mlp.fc2.bias', 'model.visual.blocks.11.mlp.fc2.weigh

...

❌ Generation failed: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

❌ Failed: Error: Failed to generate caption

🖼️ Processing: ISH.PH03.05C.004.jpg
Warning during CUDA reset: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

Attempt 1: Using prompt length 76
Template text length: 227
Vocab size: 151643
Token range: 13 to 151655
⚠️ Invalid token 151655 found! Fixing...
✅ Tokens validated: 0 to 134274
❌ Generation failed: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some oth

Safe processing: 100%|█████████▉| 749/752 [01:12<00:00, 11.23it/s]

❌ Generation failed: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

❌ Failed: Error: Failed to generate caption

🖼️ Processing: ISH.PH03.05C.006.jpg
Resized image to (896, 1024)
Warning during CUDA reset: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

Attempt 1: Using prompt length 76
Template text length: 227
Vocab size: 151643
Token range: 13 to 151655
⚠️ Invalid token 151655 found! Fixing...
✅ Tokens validated: 0 to 134274
❌ Generation failed: CUDA error: device-side assert triggered
CUDA kernel errors might be asynch

Safe processing: 100%|██████████| 752/752 [01:12<00:00, 10.38it/s]

💾 Progress saved to /content/drive/MyDrive/ImageVal/ultra_safe_captions_temp_750.csv

🖼️ Processing: ISH.PH03.05C.005.jpg
Warning during CUDA reset: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

Attempt 1: Using prompt length 76
Template text length: 227
Vocab size: 151643
Token range: 13 to 151655
⚠️ Invalid token 151655 found! Fixing...
✅ Tokens validated: 0 to 134274
❌ Generation failed: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

❌ Failed: Error: Failed to generate caption

🖼️ Processing: ISH.PH03.05B.108.jpg
War